# Data Science Essentials: Data Cleaning
    Benj McMullin
    Math 403
    10/31/2023
    

In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
import copy

### Problem 1

The g\_t\_results.csv file is a set of parent-reported scores on their child's Gifted and Talented tests. 
The two tests, OLSAT and NNAT, are used by NYC to determine if children are qualified for gifted programs.
The OLSAT Verbal has 16 questions for Kindergardeners and 30 questions for first, second, and third graders.
The NNAT has 48 questions. Each test assigns 1 point to each question asked (so there are no non integer scores).
Using this dataset, answer the following questions.



1) What column has the highest number of null values and what percent of its values are null? Print the answer as a tuple with (column name, percentage). Make sure the second value is a percent.

2) List the columns that should be numeric that aren't. Print the answer as a tuple.

3) How many third graders have scores outside the valid range for the OLSAT Verbal Score? Print the answer

4) How many data values are missing (NaN)? Print the number.


In [3]:
# Load the dataset
df = pd.read_csv('g_t_results.csv')

# 1. What column has the highest number of null values and what percent of its values are null?
null_counts = df.isnull().sum()
max_null_column = null_counts.idxmax()
percentage_null = (null_counts[max_null_column] / len(df)) * 100
answer_1 = (max_null_column, percentage_null)
print("1. Column with the highest number of null values:", answer_1)

# 2. List the columns that should be numeric but aren't.
output = ("OLSAT Verbal Score", "OLSAT Verbal Percentile", "NNAT Non Verbal Raw Score")
print("\nPart Two")
print("2. Columns that should be numeric but aren't:", output)

# 3. How many third graders have scores outside the valid range for the OLSAT Verbal Score?
third_graders = df[df['Entering Grade Level'] == '3']
valid_range_olsat_verbal = ('0', '30')
invalid_olsat_verbal_scores = third_graders[
    ~third_graders['OLSAT Verbal Score'].between(valid_range_olsat_verbal[0], valid_range_olsat_verbal[1])
]
answer_3 = len(invalid_olsat_verbal_scores)
print("3. Third graders with scores outside the valid range for OLSAT Verbal Score:", answer_3)

# 4. How many data values are missing (NaN)?
missing_data_count = df.isna().sum().sum()
print("4. Number of missing data values (NaN):", missing_data_count)

1. Column with the highest number of null values: ('School Assigned', 76.06837606837607)

Part Two
2. Columns that should be numeric but aren't: ('OLSAT Verbal Score', 'OLSAT Verbal Percentile', 'NNAT Non Verbal Raw Score')
3. Third graders with scores outside the valid range for OLSAT Verbal Score: 1
4. Number of missing data values (NaN): 193


### Problem 2

imdb.csv contains a small set of information about 99 movies. Clean the data set by doing the following in order: 

1) Remove duplicate rows by dropping the first **or** last. Print the shape of the dataframe after removing the rows.

2) Drop all rows that contain missing data. Print the shape of the dataframe after removing the rows.

3) Remove rows that have data outside valid data ranges and explain briefly how you determined your ranges for each column.

4) Identify and drop columns with three or fewer different values. Print a tuple with the names of the columns dropped.

5) Convert the titles to all lower case.

Print the first five rows of your dataframe.

In [7]:
# Load the dataset
df = pd.read_csv('imdb.csv')

# 1. Remove duplicate rows by dropping the first or last
df = df.drop_duplicates(keep='first')
print("Shape after removing duplicate rows:", df.shape)

# 2. Drop all rows that contain missing data
df = df.dropna()
print("Shape after removing rows with missing data:", df.shape)

# 3. Remove rows that have data outside of valid data ranges
# Assume valid data ranges as follows (you can adjust these as needed):
valid_duration = (30, None)  # At least 30 minutes long
valid_imdb_score = (0, None)  # Positive imdb_score
valid_title_year = (2000, None)  # Title year after 2000
df = df[
    (df['duration'] >= valid_duration[0]) &
    (df['imdb_score'] >= valid_imdb_score[0]) &
    (df['title_year'] >= valid_title_year[0])
]
print("Shape after removing rows with data outside valid ranges:", df.shape)

# 4. Identify and drop columns with three or fewer different values
columns_to_drop = [col for col in df.columns if df[col].nunique() <= 3]
df.drop(columns=columns_to_drop, inplace=True)
print("Columns dropped:", tuple(columns_to_drop))

# 5. Convert the titles to all lower case
df['movie_title'] = df['movie_title'].str.lower()

# Print the first five rows of the cleaned dataframe
print(df.head())

Shape after removing duplicate rows: (93, 13)
Shape after removing rows with missing data: (64, 13)
Shape after removing rows with data outside valid ranges: (60, 13)
Columns dropped: ('color', 'language')
       director_name  duration        gross  \
0    Martin Scorsese       240  116866727.0   
1        Shane Black       195  408992272.0   
2  Quentin Tarantino       187   54116191.0   
3   Kenneth Lonergan       186      46495.0   
4      Peter Jackson       186  258355354.0   

                                 genres                          movie_title  \
0          Biography|Comedy|Crime|Drama              the wolf of wall street   
1               Action|Adventure|Sci-Fi                           iron man 3   
2  Crime|Drama|Mystery|Thriller|Western                    the hateful eight   
3                                 Drama                             margaret   
4                     Adventure|Fantasy  the hobbit: the desolation of smaug   

   title_year country       bu

### Problem 3
Load housing.csv into a dataframe with index=0. Descriptions of the features are in housing_data_description.txt for your convenience.  
The goal is to construct a regression model that predicts SalePrice using the other features of the dataset.  Do this as follows:

	1) Identify and handle the missing data.  Hint: Dropping every row with some missing data is not a good choice because it gives you an empty dataframe.  What can you do instead?
    
    2) Add two new features: 
		a) Remodeled: Whether or not a house has been remodeled with a Y if it has been
		   remodeled, or a N if it has not.
		
		b) TotalPorch: Using the 5 different porch/deck columns, create a new column that
		   provides the total square footage of all the decks and porches for each house.
    
	3) Identify the variable with nonnumeric values that are misencoded as numbers.  One-hot encode it. Hint: don't forget to remove one of the encoded columns to prevent collinearity with the constant column (which you will add later).
    
    4) Add a constant column to the dataframe.

    5) Save a copy of the dataframe.

	6) Choose four categorical featrues that seem very important in predicting SalePrice. One-hot encode these features and remove all other categorical features.
		
	7) Run an OLS using all numerical data regression on your model.  

	
Print the ten features that have the highest coef in your model. Then print the summary. Don't print the OLS

In [7]:
# Load the dataset
house = pd.read_csv('housing.csv')

# 1. Drop all columns with more than 100 missing values
nan = house.isnull().sum()
drop = nan[nan > 100].index
house.drop(columns=drop, inplace=True)

# 2. Drop all rows with any missing values
house.dropna(inplace=True)

# 3. Drop all columns with only one unique value
house['Remodeled'] = 'N'
house.loc[house['YearRemodAdd'] > house['YearBuilt'], 'Remodeled'] = 'Y'

house['TotalPorch'] = house['WoodDeckSF'] + house['OpenPorchSF'] + house['EnclosedPorch'] + house['3SsnPorch'] + house['ScreenPorch']

# 4. Create a new column called 'Remodeled' that is 'Y' if YearRemodAdd is greater than YearBuilt and 'N' otherwise
house = pd.get_dummies(house, columns=['MSSubClass'], dtype='int64')

# 5. Create a new column called 'TotalPorch' that is the sum of WoodDeckSF, OpenPorchSF, EnclosedPorch, 3SsnPorch, and ScreenPorch
housing_cpy = house.copy()

# 6. Create dummy variables for the following columns: MSSubClass
house = pd.get_dummies(house, columns=['SaleCondition', 'ExterCond', 'BldgType', 'Neighborhood'], dtype='int64')

# 7. Create dummy variables for the following columns: SaleCondition, ExterCond, BldgType, and Neighborhood
house = house.select_dtypes(exclude=['object'])

# 8. Create a new column called 'SalePrice' that is the log of the SalePrice column
house.drop(columns=['SalePrice'], inplace=True)
house = sm.add_constant(house)
house['SalePrice'] = housing_cpy['SalePrice']
house.dropna(inplace=True)
model = sm.OLS(house['SalePrice'], house.drop(columns=['SalePrice']))

# 9. Create a new column called 'SalePrice' that is the log of the SalePrice column
results = model.fit()
df_html = results.summary().tables[1].as_html()
df = pd.read_html(df_html, header=0, index_col=0)[0]

# 10. Print the top 10 most significant variables sorted by their coefficient
df.sort_values(by='coef', ascending=False, inplace=True)
display(df.head(10))

C:\Users\benja\AppData\Local\Temp\ipykernel_3168\1434290423.py:40: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(df_html, header=0, index_col=0)[0]


,coef,std err,t,P>|t|,[0.025,0.975]
Neighborhood_StoneBr,48520.0000,32700.000,1.482,0.139,-15700.000,113000.0
Neighborhood_NridgHt,39470.0000,32400.000,1.219,0.223,-24100.000,103000.0
Neighborhood_NoRidge,34950.0000,32400.000,1.080,0.280,-28500.000,98400.0
GarageCars,15280.0000,2933.129,5.211,0.000,9529.110,21000.0
OverallQual,13820.0000,1272.377,10.865,0.000,11300.000,16300.0
MSSubClass_20,10120.0000,49700.000,0.204,0.839,-87400.000,108000.0
Neighborhood_Veenker,9546.0889,32500.000,0.294,0.769,-54200.000,73300.0
BsmtFullBath,8308.0905,2552.346,3.255,0.001,3300.755,13300.0
MSSubClass_30,8090.8279,49600.000,0.163,0.870,-89200.000,105000.0
MSSubClass_40,7831.3905,52100.000,0.150,0.881,-94400.000,110000.0


### Problem 4

Using the copy of the dataframe you created in Problem 3, one-hot encode all the categorical variables.
Print the shape of the dataframe and run OLS.

Print the ten features that have the highest coef in your model and the summary.
Write a couple of sentences discussing which model is better and why.

In [10]:
# Load the dataset
encode_columns = housing_cpy.select_dtypes(exclude=['int64', 'float64']).columns.tolist()
housing_cpy = pd.get_dummies(housing_cpy, columns=encode_columns, dtype='int64')

# Step 1: Remove columns with excessive missing values
results = sm.OLS(housing_cpy['SalePrice'], housing_cpy.drop(columns=['SalePrice'])).fit()

df_html = results.summary().tables[1].as_html()
df = pd.read_html(df_html, header=0, index_col=0)[0]

# Step 2: Sort the features by their coefficients
df.sort_values(['coef'], ascending=False, inplace=True)
display(df.head(10))

C:\Users\benja\AppData\Local\Temp\ipykernel_3168\962585096.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(df_html, header=0, index_col=0)[0]


,coef,std err,t,P>|t|,[0.025,0.975]
RoofMatl_Membran,148200.0,32300.00,4.593,0.000,84900.000,211000.0
RoofMatl_Metal,125200.0,31100.00,4.031,0.000,64300.000,186000.0
RoofMatl_WdShngl,96950.0,21100.00,4.593,0.000,55500.000,138000.0
GarageQual_Ex,83370.0,36600.00,2.279,0.023,11600.000,155000.0
Condition2_PosA,79230.0,38800.00,2.040,0.042,3013.848,155000.0
RoofStyle_Shed,63630.0,35900.00,1.770,0.077,-6890.275,134000.0
RoofMatl_Tar&Grv,55340.0,22100.00,2.499,0.013,11900.000,98800.0
RoofMatl_Roll,50500.0,30200.00,1.670,0.095,-8848.812,110000.0
RoofMatl_CompShg,44100.0,19100.00,2.313,0.021,6693.554,81500.0
Neighborhood_StoneBr,37930.0,7924.35,4.786,0.000,22400.000,53500.0
